In [31]:
# Requisitos: pandas, plotly
# Exemplo: pip install pandas plotly

import re
import pandas as pd
import plotly.graph_objects as go
import os

# caminho do CSV (fornecido por você)
csv_path = "dados-geracao-injecao.csv" # Corrigido: o arquivo está na raiz do ambiente, não em /mnt/data/

# --- leitura
df = pd.read_csv(csv_path)

# --- identificar colunas "normalizadas" pelo nome (heurística)
norm_pattern = re.compile(r"(norm(aliz)?ed|normalizad[ao]|_norm(al)?|_normalized|_scaled|minmax|zscore)", re.IGNORECASE)

# selecionar apenas colunas numéricas
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# detectar colunas numéricas que parecem normalizadas
normalized_cols = [c for c in numeric_cols if norm_pattern.search(c)]

# colunas a usar = numéricas sem as normalizadas detectadas
cols_to_use = [c for c in numeric_cols if c not in normalized_cols]

# Segurança: se sobrar < 2 colunas (não dá correlação), relaxar a exclusão:
if len(cols_to_use) < 2:
    relaxed_pattern = re.compile(r"(norm|normalized|normalizado)", re.IGNORECASE)
    normalized_cols_relaxed = [c for c in numeric_cols if relaxed_pattern.search(c)]
    cols_to_use = [c for c in numeric_cols if c not in normalized_cols_relaxed]
    if len(cols_to_use) < 2:
        # fallback: usar todas as numéricas (aviso)
        cols_to_use = numeric_cols.copy()

# --- calcular correlação
corr = df[cols_to_use].corr()

# --- criar heatmap interativo com Plotly
fig = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=corr.columns,
    y=corr.index,
    zmin=-1, zmax=1,
    colorscale="RdBu",             # diverging vermelho-azul
    colorbar=dict(title="Correlação"),
    text=corr.round(2).astype(str).values,  # valores como texto
    hovertemplate="X: %{x}<br>Y: %{y}<br>Corr: %{z:.2f}<extra></extra>"
))

fig.update_layout(
    title="Mapa de calor (heatmap) da correlação entre variáveis",
    xaxis_nticks=len(corr.columns),
    yaxis_nticks=len(corr.index),
    width=1000,
    height=900
)

# --- salvar HTML interativo
output_html = "heatmap_plotly.html" # Corrigido: salvar no diretório atual
fig.write_html(output_html, include_plotlyjs="cdn")
print(f"Heatmap interativo salvo em: {output_html}")

# --- mostrar figura (se estiver em Jupyter ou outro ambiente que suporte render)
fig.show()

# --- resumo (impressão)
print("\nResumo:")
print(f"- número de colunas numéricas detectadas: {len(numeric_cols)}")
print(f"- colunas detectadas como normalizadas (excluídas): {normalized_cols}")
print(f"- colunas usadas no cálculo da correlação: {cols_to_use}")


Heatmap interativo salvo em: heatmap_plotly.html



Resumo:
- número de colunas numéricas detectadas: 0
- colunas detectadas como normalizadas (excluídas): []
- colunas usadas no cálculo da correlação: []
